# Recommendations overlap

Here check how different are recommendations produced by our models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
from itertools import combinations

sys.path.append("../../src")
from common import config
import helpers

In [3]:
paths = config.Paths(dataset_name="jobs_published", target_users_name="all")
topK = 10
models_to_evaluate = ["als", "prod2vec", "rp3beta", "slim", "lightfm"]

In [4]:
models_recommendations = helpers.get_recommendations(
    recommendations_path=paths.recommendations_folder,
    models_to_evaluate=models_to_evaluate,
)

In [5]:
# data preparation
for model in models_recommendations:
    models_recommendations[model] = pd.concat(
        [
            models_recommendations[model].iloc[:, 0],
            models_recommendations[model]
            .iloc[:, 1 : topK + 1]
            .apply(lambda x: [i for i in x if not pd.isna(i)], axis=1),
        ],
        axis=1,
    )
    models_recommendations[model].columns = ["user", "items"]
    models_recommendations[model] = models_recommendations[model].set_index("user")

In [6]:
pd.DataFrame(
    [
        [
            model1,
            model2,
            helpers.overlap(
                models_recommendations[model1], models_recommendations[model2]
            ),
        ]
        for model1, model2 in list(combinations(list(models_recommendations.keys()), 2))
    ],
    columns=["model1", "model2", "overlap"],
)

,model1,model2,overlap
0,lightfm,als,0.374715
1,lightfm,rp3beta,0.378607
2,lightfm,slim,0.349942
3,lightfm,prod2vec,0.281287
4,als,rp3beta,0.528239
5,als,slim,0.503598
6,als,prod2vec,0.378768
7,rp3beta,slim,0.725270
8,rp3beta,prod2vec,0.370590
9,slim,prod2vec,0.347677
